# Fetch and load PDB
This code acquires the pdb file from the RCSB databank, loads the necessary information, and saves copies for possible use in visualization in other software.

In [ ]:
from pyCapsid.PDB import getCapsid
pdb = '7kq5'
capsid, calphas, coords, bfactors, chain_starts, title = getCapsid(pdb, save=True)

# Build ENM Hessian
This code builds a hessian matrix using an elastic network model defined by the given parameters. The types of model and the meaning of the parameters are provided in the documentation.

In [ ]:
from pyCapsid.ENM import buildENMPreset
kirch, hessian = buildENMPreset(coords, preset='U-ENM')

# Perform NMA
## Calculate low frequency modes
This code calculates the n lowest frequency modes of the system by calculating the eigenvalues and eigenvectors of the hessian matrix.

In [ ]:
from pyCapsid.NMA import modeCalc

evals, evecs = modeCalc(hessian)

## Predict, fit, and compare b-factors
This code uses the resulting normal modes and frequencies to predict the b-factors of each alpha carbon, fits these results to experimental values from the pdb entry, and plots the results for comparison.

In [ ]:
from pyCapsid.NMA import fitCompareBfactors
evals_scaled, evecs_scaled = fitCompareBfactors(evals, evecs, bfactors, pdb, fitModes=False)

# Perform quasi-rigid cluster identification (QRC)

In [ ]:
from pyCapsid.NMA import calcDistFlucts

dist_flucts = calcDistFlucts(evals_scaled, evecs, coords)

In [ ]:
from pyCapsid.QRC import findQuasiRigidClusters
import numpy as np
n_cluster_max = 62
n_range = np.arange(4, n_cluster_max, 2)
labels, score  = findQuasiRigidClusters(pdb, dist_flucts, n_range)

## Visualize in jupyter notebook with nglview
You can visualize the results in the notebook with nglview. The following function returns an nglview object with the results colored based on cluster. See the nglview documentation for further info (http://nglviewer.org/nglview/release/v2.7.7/index.html)

In [ ]:
from pyCapsid.viz_util import view_pdb_ngl
view = view_pdb_ngl(pdb, capsid, labels)
view

## Visualize in ChimeraX
If ChimeraX (https://www.cgl.ucsf.edu/chimerax/download.html) you may provide a path to the chimerax.exe file to automatically visualize the results in chimerax.

In [ ]:
from pyCapsid.viz_util import chimeraxViz
chimeraxViz(labels, pdb, chimerax_path='C:\\Program Files\\ChimeraX\\bin')